# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import os
import sys
sys.path.append('../../python_tools/')
from tools import *

In [3]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load and edit files

LAM output analysis on native ICO grid using tools from Valou and Cécile Agosta.

## Sims

In [4]:
LAM_dir='../../../JZ_simu_outputs/LAM/native_outputs/'

In [5]:
import psyplot.project as psy
psy.rcParams['auto_show'] = True
mpl.rcParams['figure.figsize'] = [10., 8.]

orc_file='{}/SRF/Output/MO/sim254_20170701_20170731_1M_sechiba_history.nc'.format(LAM_dir)
diag_routing_file='{}/SRF/Output/MO/sim254_20170701_20170731_1M_diag_routing.nc'.format(LAM_dir)
lmdz_file='{}/ATM/Output/MO/sim254_20170701_20170731_1M_histmth.nc'.format(LAM_dir)
restartphy_file='{}/ATM/Restart/sim254_20170731_restartphy.nc'.format(LAM_dir)
restart_file='{}/ICO/Restart/sim254_20170731_restart.nc'.format(LAM_dir)
start_file='{}/espagne_start_2010.nc'.format(LAM_dir)

In [ ]:
ORC = xr.open_mfdataset(orc_file)
ORC = ORC.rename({'time_counter':'time'})
ORC

In [ ]:
LMDZ=xr.open_mfdataset(lmdz_file)
LMDZ = LMDZ.rename({'time_counter':'time'})
LMDZ

In [ ]:
restartphy=xr.open_dataset(restartphy_file)
restartphy

In [ ]:
restart=xr.open_dataset(restart_file)
restart

In [ ]:
start=xr.open_dataset(start_file)
start

## Changes to dataset

In [11]:
#make new var in lmdz netcdf file equal to precip *3600*24
LMDZ['precip_mm'] = LMDZ['precip']*3600*24
#save to netcdf file
lmdz_file2='{}/ATM/Output/MO/sim254_20170701_20170731_1M_histmth_modified.nc'.format(LAM_dir)
LMDZ.to_netcdf(lmdz_file2)

In [12]:
orc_file2='{}/SRF/Output/MO/sim254_20170701_20170731_1M_sechiba_history_modified.nc'.format(LAM_dir)
os.system('rm -f {}'.format(orc_file2))

ORC['soiltile_1']=ORC['soiltile'][0,:]
ORC['soiltile_2']=ORC['soiltile'][1,:]
ORC['soiltile_3']=ORC['soiltile'][2,:]

ORC.to_netcdf(orc_file2)

In [ ]:
start["Altitude"] = start["phis"]/9.81
start["Altitude"].attrs["units"] = "m"
start["Altitude"].attrs["long_name"] = "Altitude"
start["Altitude"].attrs["standard_name"] = "Altitude"

start_file2='{}/espagne_start_2010_modified.nc'.format(LAM_dir)
start.to_netcdf(start_file2)

# Maps

In [ ]:
file=orc_file
var='Areas'
vmin=0.0
vmax=2000
pas=(vmax-vmin) / 10

map=psy.plot.mapplot(
    file, 
    name=var,
    datagrid=dict(color='k', linewidth=0.2),
    cbar='r',
    tight=True,
    lsm='50m',
    cmap=myvir,
    extend='both',
    projection='ortho',
    xgrid=True,ygrid=True,
    title=var,
    bounds=np.arange(vmin, vmax + pas, pas),
    clabel="Altitude (m)"
)
# map.cbar.set_label('Altitude (meters)')

In [ ]:
psy.plot.show_fmtkeys('mapplot')

In [ ]:
ORC['Areas']

In [ ]:
import psyplot.project as psy
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import numpy as np
import xarray as xr

import cartopy
import cartopy.util as cu
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


psy.rcParams['plotter.maps.xgrid'] = False
psy.rcParams['plotter.maps.ygrid'] = False
mpl.rcParams['figure.figsize'] = [10., 8.]

lon=0
lat=40

maps=psy.plot.mapplot(ORC, name='Areas', datagrid=dict(color='k', linewidth=0.2), 
                      cbar='r',clon=lon,clat=lat,tight=True,
                      bounds=np.linspace(0,25000,21,endpoint=True),lsm='50m',projection='ortho',
                      cmap='terrain',xgrid=True,ygrid=True)

In [ ]:
file=diag_routing_file
var='routing_irrigation'
vmin=0.0
vmax=0.01
pas=(vmax-vmin) / 10

psy.plot.mapplot(
    file, 
    name=var,
    datagrid=dict(color='k', linewidth=0.2),
    cbar='r',
    tight=True,
    lsm='50m',
    cmap=wetW,
    extend='both',
    projection='ortho',
    #add title
    title=var,
    # bounds=np.arange(vmin, vmax + pas, pas)
)

In [ ]:
file=lmdz_file
var='evap'
vmin=0.0
vmax=0.5e+6
pas=0.001e+6

psy.plot.mapplot(
    file, 
    name=var,
    datagrid=dict(color='k', linewidth=0.2),
    cbar='r',
    tight=True,
    lsm='50m',
    cmap=reds,
    extend='both',
    projection='ortho',
    title=var,
    # bounds=np.arange(vmin, vmax + pas, pas)
)